# Step 0. Definition

In [ ]:
#import dtnmonitor_v2 as dtn
import dtn_module.dtnmonitor_v3  as DtnModule
import os,sys
import getpass
import subprocess

dtn = DtnModule.DTNMonitor()

user=getpass.getuser()

DtnModule.show_interface()

DtnModule.show_nvme()

## this is necessary to show exec info in notebook
def exec_print(command):
    process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
    for line in iter(process.stdout.readline, b''):
        # system print
        sys.stdout.write(line)   

# Step 1. Parameter Setting

In [ ]:
########## Necessary
###  SELECT THE MONITOR NETWORK INTERFACE
## used in Step 1,2,3 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
dtn.interface = 'all'
#dtn.interface = 'all'


### P3. NUTTCP THREADS
## used in Step 2 :input threads number
#count = 8

## display the time period
# [0,100]: 100 means display current 100 secs, 0=all
dtn.max_graph_point=100


file_write_nuttcp=[
"/data/disk0/sc17/fftest",
"/data/disk1/sc17/fftest",
"/data/disk2/sc17/fftest",
"/data/disk3/sc17/fftest",
"/data/disk4/sc17/fftest",
"/data/disk5/sc17/fftest",
"/data/disk6/sc17/fftest",
"/data/disk7/sc17/fftest",
    # "/foo/bar/filennn"
     ]


In [ ]:
# setup fio parameters
# first, filename is assigned with existed and >1GB file path 
randrepeat=" --randrepeat=0" #--randrepeat=0 
bs=" --bs=1M" #--bs=1M 
ioengine=" --ioengine=sync "#--ioengine=sync 
tuntime=" --runtime=30" #--runtime=30 
iodepth=" --iodepth=32"#--iodepth=32 
name=" --name=drive0"#--name=drive0 
size=" --size=100G "#--size=100G 





# Step 2. Start servers for network transfer by memory

In [ ]:
## in memory

count= len(file_write_nuttcp)

for i in range(file_write_nuttcp):
    command = "nuttcp -S -1 -P 5200"+str(i)+" -p 5210"+str(i)+ " &"
    #print(command)
    os.system(command)



# Step 3. Start servers for network transfer by disk

In [ ]:
## in disk

count= len(file_write_nuttcp)


for i in range(count):
    command = "nuttcp -S -1 -sdz -P 5300"+str(i)+" -p 5310"+str(i)+ " > "+ file_write_nuttcp[i] +" & "
    #print(command)
    os.system(command)
    #dtn.exec_sysout(command)



In [ ]:
exec_print("sleep 3; ps -u "+ user +" ux |grep nuttcp")

# Step 4. Start Monitor

In [ ]:
dtn.run_monitor(timeout=60)

## Step A . stop the test nuttcp

In [ ]:
dtn.exec_sysout("killall -u "+user+" nuttcp")

In [ ]:
exec_print("sleep 3; ps -u "+ user +" ux |grep nuttcp")